# Lazypredict: Запуск усіх алгоритмів `sklearn` одним рядком коду

Як його використовувати (і чому не варто)

https://pub.towardsai.net/lazypredict-run-all-sklearn-algorithms-with-a-line-of-code-29d73d82499c

In [1]:
from lazypredict.Supervised import LazyRegressor
from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np
import pandas as pd


## Спочатку ми імпортуємо набір даних "Діабет".

Для кожного з $n = 442$ пацієнтів з діабетом було отримано десять базових змінних: вік, стать, індекс маси тіла, середній кров'яний тиск та шість показників сироватки крові, а також відповідь, що нас цікавить, - кількісний показник прогресування хвороби через рік після базового рівня.

In [2]:
# Import the Diabetes Dataset
diabetes = datasets.load_diabetes()

print(diabetes.DESCR)


.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

## Далі ми перетасуємо набір даних, щоб розділити їх на набори тренувань і тестів.

In [3]:
# Shuffle the dataset
X, y = shuffle(diabetes.data, diabetes.target, random_state=13)

# Cast the numerical values into a numpy float.
X = X.astype(np.float32)

# Split the dataset into 90% and 10%.
offset = int(X.shape[0] * 0.9)

# Split into train and test
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]


## Далі ініціалізуємо об'єкт LazyRegressor

In [4]:
# Running the Lazypredict library and fit multiple regression libraries
# for the same dataset
reg = LazyRegressor(verbose=0,
                    ignore_warnings=False,
                    custom_metric=None,
                    predictions=False,
                    random_state=13,
                    )

# Parameters
# ----------
# verbose : int, optional (default=0)
#       For the liblinear and lbfgs solvers set verbose to any positive
#       number for verbosity.
# ignore_warnings : bool, optional (default=True)
#       When set to True, the warning related to algorigms that are not able
#       to run are ignored.
# custom_metric : function, optional (default=None)
#       When function is provided, models are evaluated based on the custom
#       evaluation metric provided.
# prediction : bool, optional (default=False)
#       When set to True, the predictions of all the models models are
#       returned as dataframe.
# regressors : list, optional (default="all")
#       When function is provided, trains the chosen regressor(s).


## Тепер ми поєднаємо алгоритми множинної регресії з бібліотекою `lazypredict`. Цей крок зайняв 3 секунди.

По суті, метод підгонки робить наступне:

1. Розділіть всі об'єкти на три категорії: числові (об'єкти, які є числами) або категоріальні (об'єкти, які є текстом). 
2. Далі розділіть категоріальні ознаки на дві: "високі" категоріальні ознаки (які мають більше унікальних значень, ніж загальна кількість ознак) і "низькі" категоріальні ознаки (які мають менше унікальних значень, ніж загальна кількість ознак).
3. Потім кожна ознака попередньо обробляється таким чином:

- Числові ознаки: Замінимо відсутні значення середнім значенням, а потім стандартизуємо ознаку (вилучимо середнє значення та поділити на дисперсію) 
- "Високі" категоріальні ознаки: Пропущені значення замінюються значенням "missing", а потім виконується однократне кодування. 
- "Низькі" категоріальні ознаки: Присвоїмо відсутнім значенням значення "missing", а потім виконати порядкове кодування (перетворити кожне унікальне значення рядка на ціле число). У прикладі зі стовпчиком "Стать" - "Чоловік" кодується як 0, а "Жінка" - як 1.
- Підберемо навчальний набір даних для кожного алгоритму. - 
- Протестуємо кожен алгоритм на тестовому наборі даних. За замовчуванням, метрики наступні: приведений $R^2$, $R^2$, середньоквадратична помилка (MSE) та витрачений час.

In [5]:
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
model_dictionary = reg.provide_models(X_train, X_test, y_train, y_test)


100%|██████████| 42/42 [00:05<00:00,  7.34it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000089 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 640
[LightGBM] [Info] Number of data points in the train set: 397, number of used features: 10
[LightGBM] [Info] Start training from score 151.722922
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [6]:
models


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
OrthogonalMatchingPursuitCV,0.37,0.52,54.39,0.02
Lasso,0.37,0.52,54.46,0.02
LassoLars,0.37,0.52,54.46,0.02
LarsCV,0.37,0.51,54.54,0.03
LassoCV,0.37,0.51,54.59,0.08
SGDRegressor,0.37,0.51,54.79,0.01
LassoLarsIC,0.36,0.51,54.83,0.02
RidgeCV,0.36,0.51,54.91,0.01
Ridge,0.36,0.51,54.91,0.01


## Чи рекомендую я `lazypredict`?

Якщо ви встановите його, `lazypredict` дуже простий у використанні. Його синтаксис дуже близький до `scikit-learn`, що робить криву навчання дуже пологою.

Але у нього є кілька критичних недоліків.

1. __Складне встановлення__: Багато хто повідомляв про труднощі при встановленні бібліотек, тому що розробники не додали файл requirements.txt, який документує необхідні залежності.
   
2. __Обмежена документація__: Мені довелося прочісувати вихідний код, щоб дізнатися, як працює препроцесор. Це не ідеально. Я також не знаю гіперпараметрів, які використовуються для виконання кожного із завдань класифікації та регресії. 
   
3. __Обмежена можливість налаштування__: Я все ще не знайшов способів налаштувати кроки попередньої обробки. 
   
4. Незрозуміло, як використовувати модель після `lazypredict`: Після того, як ви закінчите роботу з бібліотекою `lazypredict`, ви в ідеалі захочете вибрати найкращий алгоритм. `Lazypredict` не робить це простим, оскільки у вас немає простого способу експортувати найкращий алгоритм.